In [ ]:
import requests
import json
import csv
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio

In [ ]:
def fetch_all_books(topic="children"):
    url = "https://gutendex.com/books"
    params = {"topic": topic}
    all_books = []

    while url:
        response = requests.get(url, params=params if url == "https://gutendex.com/books" else None)
        data = response.json()
        all_books.extend(data["results"])
        url = data["next"]  # Следующая страница

    return all_books

def extract_book_data(books):
    data = []
    for book in books:
        authors = book.get("authors", [])
        first_author = authors[0] if authors else {"name": "N/A", "birth_year": "N/A", "death_year": "N/A"}

        data.append({
            "title": book["title"],
            "author_name": first_author["name"],
            "author_birth_year": first_author.get("birth_year", "N/A"),
            "author_death_year": first_author.get("death_year", "N/A"),
            "translators": ", ".join([t["name"] for t in book.get("translators", [])]),
            "subjects": ", ".join(book.get("subjects", [])),
            "bookshelves": ", ".join(book.get("bookshelves", [])),
            "copyright": book.get("copyright"),
            "download_count": book.get("download_count")
        })

    return pd.DataFrame(data)

books = fetch_all_books()
df = extract_book_data(books)
df.to_csv("children_books.csv", index=False)
print("Dataset saved as children_books.csv")

Dataset saved as children_books.csv


In [ ]:
df = pd.read_csv("children_books.csv")

# Выведем количество строк в датафрейме
print(f"Количество строк в файле children_books.csv: {len(df)}")

Количество строк в файле children_books.csv: 8999


In [ ]:
def fetch_all_books(topic=None):
    url = "https://gutendex.com/books"
    params = {"topic": topic} if topic else {}
    all_books = []

    while url:
        response = requests.get(url, params=params if url == "https://gutendex.com/books" else None)
        data = response.json()
        all_books.extend(data["results"])
        url = data["next"]  # Следующая страница

    return all_books

def extract_book_data(books):
    data = []
    for book in books:
        authors = book.get("authors", [])
        first_author = authors[0] if authors else {"name": "N/A", "birth_year": "N/A", "death_year": "N/A"}

        data.append({
            "title": book["title"],
            "author_name": first_author["name"],
            "author_birth_year": first_author.get("birth_year", "N/A"),
            "author_death_year": first_author.get("death_year", "N/A"),
            "translators": ", ".join([t["name"] for t in book.get("translators", [])]),
            "subjects": ", ".join(book.get("subjects", [])),
            "bookshelves": ", ".join(book.get("bookshelves", [])),
            "copyright": book.get("copyright"),
            "download_count": book.get("download_count")
        })

    return pd.DataFrame(data)

# Fetch all books excluding children
books = fetch_all_books(topic="")  # Пустая строка или None получит все книги без конкретной темы
df = extract_book_data(books)
df.to_csv("non_children_books.csv", index=False)
print("Dataset saved as non_children_books.csv")

Dataset saved as non_children_books.csv
